In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dpl-disaster-tweets/sample_submission.csv
/kaggle/input/dpl-disaster-tweets/train.csv
/kaggle/input/dpl-disaster-tweets/test.csv


In [3]:
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [41]:
import pandas as pd
dataset = pd.read_csv('/kaggle/input/dpl-disaster-tweets/train.csv')
dataset.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [42]:
df = dataset.drop(['keyword', 'location'], axis=1)
df.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [43]:
df.isna().sum()

id        0
text      0
target    0
dtype: int64

In [44]:
len(df[df['target']==1])

3271

In [45]:
len(df[df['target']==0])

4342

In [46]:
df['text']=df['text'].apply(lambda x: x.lower())

In [47]:
df['text']=df['text'].apply(lambda x: x.replace("#", ""))

In [48]:
import string 
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [49]:
str.maketrans(dict.fromkeys(string.punctuation))

{33: None,
 34: None,
 35: None,
 36: None,
 37: None,
 38: None,
 39: None,
 40: None,
 41: None,
 42: None,
 43: None,
 44: None,
 45: None,
 46: None,
 47: None,
 58: None,
 59: None,
 60: None,
 61: None,
 62: None,
 63: None,
 64: None,
 91: None,
 92: None,
 93: None,
 94: None,
 95: None,
 96: None,
 123: None,
 124: None,
 125: None,
 126: None}

In [50]:
df['text']=df['text'].apply(lambda x: x.translate(str.maketrans(dict.fromkeys(string.punctuation))))

In [51]:
df['text'][5]

'rockyfire update  california hwy 20 closed in both directions due to lake county fire  cafire wildfires'

In [52]:
df['text']=df['text'].apply(lambda x: x.translate(str.maketrans('', '', string.digits)))

In [53]:
df['text'][5]

'rockyfire update  california hwy  closed in both directions due to lake county fire  cafire wildfires'

In [54]:
for index, row in df.iterrows():
    text=[]
    for word in row['text'].split():
        if "http" not in word:
            text.append(word)
    row['text']=" ".join(text)
    #print(row['text'])
    df['text'][index]=row['text']
        

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [55]:
df['text'][7612]

'the latest more homes razed by northern california wildfire abc news'

In [19]:
!pip install transformers

In [56]:
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, default_data_collator, Trainer, set_seed
set_seed(42)

In [57]:
import torch

In [38]:
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Khởi tạo tokenizer và mô hình
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForSequenceClassification.from_pretrained('bert-large-uncased')

# Xây dựng lớp mới
new_classifier = nn.Sequential(
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, model.classifier.out_features)  # Giả sử bạn muốn giữ nguyên số lượng lớp đầu ra
)

# Gán lớp mới vào mô hình
model.classifier = new_classifier

# Chuyển mô hình sang GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [58]:
maxx, minn = 0, 1000000
for index, row in df.iterrows():
    if len(row['text'].split())>maxx:
        maxx=len(row['text'].split())
    if len(row['text'].split())<minn:
        minn=len(row['text'].split())
maxx, minn

(31, 1)

In [59]:
max_length=64
df['input_ids']=df['text'].apply(lambda x: tokenizer(x, max_length=max_length, padding='max_length')['input_ids'])
df.head(n=10)

,id,text,target,input_ids
0,1,our deeds are the reason of this earthquake ma...,1,"[101, 2256, 15616, 2024, 1996, 3114, 1997, 202..."
1,4,forest fire near la ronge sask canada,1,"[101, 3224, 2543, 2379, 2474, 6902, 3351, 2187..."
2,5,all residents asked to shelter in place are be...,1,"[101, 2035, 3901, 2356, 2000, 7713, 1999, 2173..."
3,6,people receive wildfires evacuation orders in ...,1,"[101, 2111, 4374, 3748, 26332, 13982, 4449, 19..."
4,7,just got sent this photo from ruby alaska as s...,1,"[101, 2074, 2288, 2741, 2023, 6302, 2013, 1009..."
5,8,rockyfire update california hwy closed in both...,1,"[101, 6857, 10273, 10651, 2662, 1044, 18418, 2..."
6,10,flood disaster heavy rain causes flash floodin...,1,"[101, 7186, 7071, 3082, 4542, 5320, 5956, 9451..."
7,13,im on top of the hill and i can see a fire in ...,1,"[101, 10047, 2006, 2327, 1997, 1996, 2940, 199..."
8,14,theres an emergency evacuation happening now i...,1,"[101, 2045, 2015, 2019, 5057, 13982, 6230, 208..."
9,15,im afraid that the tornado is coming to our area,1,"[101, 10047, 4452, 2008, 1996, 11352, 2003, 27..."


In [60]:
df.rename(columns={'target':'labels'}, inplace=True)
df.head()

,id,text,labels,input_ids
0,1,our deeds are the reason of this earthquake ma...,1,"[101, 2256, 15616, 2024, 1996, 3114, 1997, 202..."
1,4,forest fire near la ronge sask canada,1,"[101, 3224, 2543, 2379, 2474, 6902, 3351, 2187..."
2,5,all residents asked to shelter in place are be...,1,"[101, 2035, 3901, 2356, 2000, 7713, 1999, 2173..."
3,6,people receive wildfires evacuation orders in ...,1,"[101, 2111, 4374, 3748, 26332, 13982, 4449, 19..."
4,7,just got sent this photo from ruby alaska as s...,1,"[101, 2074, 2288, 2741, 2023, 6302, 2013, 1009..."


In [61]:
df = df[['input_ids', 'labels']]
df.head()

,input_ids,labels
0,"[101, 2256, 15616, 2024, 1996, 3114, 1997, 202...",1
1,"[101, 3224, 2543, 2379, 2474, 6902, 3351, 2187...",1
2,"[101, 2035, 3901, 2356, 2000, 7713, 1999, 2173...",1
3,"[101, 2111, 4374, 3748, 26332, 13982, 4449, 19...",1
4,"[101, 2074, 2288, 2741, 2023, 6302, 2013, 1009...",1


In [62]:
train_df = df[:-int(len(df)*0.01)].reset_index(drop=True)
test_df = df[-int(len(df)*0.01):].reset_index(drop=True)

In [63]:
len(train_df), len(test_df)

(7537, 76)

In [64]:
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

In [65]:
train_ds

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 7537
})

In [68]:
batch_size = 16

args = TrainingArguments(
    'nlp-getting-started',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate=3e-5,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    warmup_ratio=0.1,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
)

data_collator = default_data_collator
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [69]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,0.458532,0.548800,138.496000


TrainOutput(global_step=59, training_loss=0.6511969485525357, metrics={'train_runtime': 200.2771, 'train_samples_per_second': 0.295, 'total_flos': 971492183200512.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 8192, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1687552, 'train_mem_gpu_alloc_delta': 2686966272, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 2030453248})

In [72]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [73]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

{'eval_loss': 0.4585317373275757,
 'eval_accuracy': 0.9210526315789473,
 'eval_runtime': 0.6227,
 'eval_samples_per_second': 122.04,
 'init_mem_cpu_alloc_delta': 0,
 'init_mem_gpu_alloc_delta': 0,
 'init_mem_cpu_peaked_delta': 0,
 'init_mem_gpu_peaked_delta': 0,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_peaked_delta': 46166016}

In [75]:
df_submission = pd.read_csv('/kaggle/input/dpl-disaster-tweets/test.csv')
df_submission.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [76]:
df_submission['text']=df_submission['text'].apply(lambda x: x.lower())

In [77]:
df_submission['text']=df_submission['text'].apply(lambda x: x.replace("#", ""))

In [78]:
df_submission['text']=df_submission['text'].apply(lambda x: x.translate(str.maketrans(dict.fromkeys(string.punctuation))))

In [79]:
df_submission['text']=df_submission['text'].apply(lambda x: x.translate(str.maketrans('', '', string.digits)))

In [80]:
for index, row in df_submission.iterrows():
    text=[]
    for word in row['text'].split():
        if "http" not in word:
            text.append(word)
    row['text']=" ".join(text)
    #print(row['text'])
    df_submission['text'][index]=row['text']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [81]:
df_submission['text'][89]

'cop pulls drunk driver to safety seconds before his car is hit by train via viralspell'

In [82]:
df_submission['input_ids']=df_submission['text'].apply(lambda x: tokenizer(x, max_length=max_length, padding='max_length')['input_ids'])
df_submission.head()

,id,keyword,location,text,input_ids
0,0,NaN,NaN,just happened a terrible car crash,"[101, 2074, 3047, 1037, 6659, 2482, 5823, 102,..."
1,2,NaN,NaN,heard about earthquake is different cities sta...,"[101, 2657, 2055, 8372, 2003, 2367, 3655, 2994..."
2,3,NaN,NaN,there is a forest fire at spot pond geese are ...,"[101, 2045, 2003, 1037, 3224, 2543, 2012, 3962..."
3,9,NaN,NaN,apocalypse lighting spokane wildfires,"[101, 16976, 7497, 21878, 3748, 26332, 102, 0,..."
4,11,NaN,NaN,typhoon soudelor kills in china and taiwan,"[101, 15393, 2061, 12672, 10626, 8563, 1999, 2..."


In [83]:
df_submission=df_submission[['input_ids']]
df_submission.head()

,input_ids
0,"[101, 2074, 3047, 1037, 6659, 2482, 5823, 102,..."
1,"[101, 2657, 2055, 8372, 2003, 2367, 3655, 2994..."
2,"[101, 2045, 2003, 1037, 3224, 2543, 2012, 3962..."
3,"[101, 16976, 7497, 21878, 3748, 26332, 102, 0,..."
4,"[101, 15393, 2061, 12672, 10626, 8563, 1999, 2..."


In [84]:
output_ds = Dataset.from_pandas(df_submission)
output_ds

Dataset({
    features: ['input_ids'],
    num_rows: 3263
})

In [85]:
outputs=trainer.predict(output_ds)

In [86]:
outputs.predictions.argmax(1)

array([0, 0, 0, ..., 0, 1, 1])

In [88]:
sub = pd.read_csv('../input/dpl-disaster-tweets/sample_submission.csv')
sub.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [89]:
sub['target'] = outputs.predictions.argmax(1)
sub.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,1


In [90]:
sub.to_csv('submission_final.csv', index=False)